In [1]:
import pandas as pd
import datetime
import numpy as np
import sys
sys.path.append('./utils')
from utils import *
from process import * 
from add_list_master import *
import japanize_matplotlib

In [2]:
# データ保存先
data_folda = '../data/'

target_column="target6"
make_genre_list=True

train_name="train_with_master_structured_main_medias.pkl"
test_name="test3_with_master_structured_main_medias.pkl"

In [3]:
train_data = pd.read_pickle(data_folda+train_name)
test_data = pd.read_pickle(data_folda+test_name)

In [4]:
# object型のカラムをリスト化する関数
def get_object_column_list(df):
    object_list = []
    for column in df.columns:
        if df[column].dtype == "object":
            object_list.append(column)
    return object_list

object_list = get_object_column_list(train_data)

In [5]:
# object 型で int に変換できるカラムをリスト化
int_convert_list = []
for column in object_list:
    if(len(train_data[column].value_counts())<20):
        print(column,train_data[column].value_counts())
        int_convert_list.append(column)

# 欠損と NaN の見分けがつかなくなることを避ける


general_rank_GOOGLE A    4568844
L    3382548
S    2001156
B     317268
N     135288
F      28152
Name: general_rank_GOOGLE, dtype: int64
infos.familiar.stars      7216200
1     262632
2     122268
3      48840
Name: infos.familiar.stars, dtype: int64
general_rank_RETTY A    3085668
L    2229756
S    2212032
B     120780
N       1008
F        696
Name: general_rank_RETTY, dtype: int64
infos.reserveEntireStore         7221408
可        318972
不可       109548
可、不可         12
Name: infos.reserveEntireStore, dtype: int64
infos.counterSeats.value      4745412
有    1916028
無     988500
Name: infos.counterSeats.value, dtype: int64
infos.privateRoom.value      4736472
無    2234484
有     678984
Name: infos.privateRoom.value, dtype: int64
general_rank_HOTPEPPER L    2559048
A    2051724
S    1412268
B     161616
N      26124
F        660
Name: general_rank_HOTPEPPER, dtype: int64
infos.couponCount 0    5279880
3     289416
2     224376
1     222552
4      71076
5      65880
6      34956
7      13

In [16]:
len(object_list)

42

In [7]:
# int_convert_list に含まれるカラムを float に変換
# general_rankは除く
# float にならないものは カテゴリ化

for column in int_convert_list:
    if column.find("general_rank") == -1:
        try:
            print(column)
            # 空文字は先に、np.Nan に変換する。クロールできなかったメディアとの区別は general_rank の有無を使う。
            train_data[column] = train_data[column].replace('',np.nan).astype(float)
            test_data[column] = test_data[column].replace('',np.nan).astype(float)
        except:
            print(column)
            train_data[column] = train_data[column].astype("category")
            test_data[column] = test_data[column].astype("category")

# infos.familiar.stars の欠損が別の値になっていないか -> 欠損も空文字もまとめて、空文字になっている。
# aggregateRating はどうなった？

infos.familiar.stars
infos.reserveEntireStore
infos.reserveEntireStore
infos.counterSeats.value
infos.counterSeats.value
infos.privateRoom.value
infos.privateRoom.value
infos.couponCount
aggregateRating.reviewCount
aggregateRating.bestRating
aggregateRating.worstRating
interactionCount.went
interactionCount.wanttogo
paymentAccepted
hostname
hostname
infos.aggregateRating_TRIPADVISOR


In [10]:
train_data["infos.counterSeats.value"]

0            
1            
2            
3            
4            
           ..
10503763     
10503764     
10503765     
10503766     
10503767     
Name: infos.counterSeats.value, Length: 10503768, dtype: category
Categories (3, object): ['', '有', '無']

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10503768 entries, 0 to 10503767
Columns: 114 entries, restaurant_id to infos.onlineReservation_HITOSARA
dtypes: bool(1), category(7), datetime64[ns](5), float32(47), float64(13), int16(3), int32(5), int8(4), object(29)
memory usage: 6.0+ GB


In [14]:
# aggregateRating.reviewCount, aggregateRating.bestRating, aggregateRating.worstRating, interactionCount.went, interactionCount.wanttogo,
# paymentAccepted, hostname, infos.aggregateRating_TRIPADVISOR はカラムから削除する
drop_list = ["aggregateRating.reviewCount","aggregateRating.bestRating","aggregateRating.worstRating","interactionCount.went","interactionCount.wanttogo","paymentAccepted","hostname","infos.aggregateRating_TRIPADVISOR"]
train_data = train_data.drop(drop_list,axis=1)
test_data = test_data.drop(drop_list,axis=1)

In [15]:
train_data.to_csv(data_folda+"train_data_fix_dtypes.csv",index=False)
test_data.to_csv(data_folda+"test_data_fix_dtypes.csv",index=False)